In [ ]:
import boto3
import configparser
import ftplib
import urllib.request
import time
import py7zr

In [ ]:
# Create a ConfigParser instance to read configuration from 'credentials.txt'
config = configparser.ConfigParser()
config.read_file(open('credentials.txt'))

In [ ]:
# Retrieve AWS access key and secret from the configuration file
KEY = config.get('AWS', 'KEY')
SECRET = config.get('AWS', 'SECRET')

In [ ]:
# Create an AWS session using the retrieved access key and secret
session = boto3.Session(
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET
)

In [ ]:
# Create an instance of the FTP class for connection to the FTP server
ftp = ftplib.FTP('ftp.mtps.gov.br')
ftp.login()


In [ ]:
# Define ranges of years for processing
anos_2010_2019 = range(2010, 2020)  # Range from 2010 to 2019
anos_2020_2023 = range(2020, 2024)  # Range from 2020 to 2023

In [ ]:
# Import necessary libraries
import urllib.request
import ftplib
import boto3

# Establish an FTP connection to the server
ftp = ftplib.FTP('ftp.mtps.gov.br')
ftp.login()

# AWS S3 client initialization
s3_client = boto3.client('s3')

# Years for different data sources
anos_2020_2023 = range(2020, 2024)  # Range of years 2020 to 2023
anos_2010_2019 = range(2010, 2020)  # Range of years 2010 to 2019

# Iterate through the years 2020 to 2023
for ano in anos_2020_2023:
    # Change to the directory of the "NOVO CAGED"
    ftp.cwd('/pdet/microdados/NOVO CAGED/')

    # Change to the directory of the year
    ftp.cwd(str(ano))

    # List the month directories in the year directory
    meses = ftp.nlst()

    # Iterate through the month directories
    for mes in meses:
        # Change to the month directory
        ftp.cwd(mes)

        # List the files in the month directory
        files = ftp.nlst()

        # Iterate through the files
        for file in files:
            if 'MOV' in file:
                # Construct the FTP file path
                ftp_file_path = f'ftp://ftp.mtps.gov.br/pdet/microdados/NOVO CAGED/{ano}/{mes}/{file}'

                # Construct the local file path
                local_file_path = f'C:/Users/bruno/Documents/Projeto Engenharia de Dados/files/{file}'

                # Download the file from FTP
                urllib.request.urlretrieve(ftp_file_path, local_file_path)

                # Upload the file to S3
                bucket_name = 'caged-bucket'  # S3 bucket name
                s3_key = f'bronze/{file}'  # S3 file path and name

                s3_client.upload_file(
                    Filename=local_file_path,
                    Bucket=bucket_name,
                    Key=s3_key
                )

                print(f"File {file} successfully uploaded to Amazon S3!")

        # Go back to the main year directory
        ftp.cwd('..')

# Iterate through the years 2010 to 2019
for ano in anos_2010_2019:
    # Change to the directory of the year
    ftp.cwd(f'/pdet/microdados/CAGED/{ano}')

    # List the files in the directory
    files = ftp.nlst()

    # Iterate through the files
    for file in files:
        # Construct the FTP file path
        ftp_file_path = f'ftp://ftp.mtps.gov.br/pdet/microdados/CAGED/{ano}/{file}'

        # Construct the local file path
        local_file_path = f'C:/Users/bruno/Documents/Projeto Engenharia de Dados/files/{file}'

        # Download the file from FTP
        urllib.request.urlretrieve(ftp_file_path, local_file_path)

        # Upload the file to S3
        bucket_name = 'caged-bucket'  # S3 bucket name
        s3_key = f'raw/{file}'  # S3 file path and name

        s3_client.upload_file(
            Filename=local_file_path,
            Bucket=bucket_name,
            Key=s3_key
        )

        print(f"File {file} uploaded to Amazon S3")

# Close the FTP connection
ftp.quit()